# 📊 Sector Rotation Strategy vs. SPY

In [ ]:
# Install required packages
!pip install --quiet pandas==2.2.2 yfinance tqdm matplotlib


In [ ]:
# Imports and setup
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.close('all')

# Strategy parameters
ETFS = ["XLY","XLP","XLE","XLF","XLV","XLI","XLB","XLK","XLU"]
START = "2015-01-01"
MOM = 6        # lookback in months
TOP_N = 3      # number of ETFs to hold
TC = 0.001     # transaction cost


In [ ]:
# Download and prepare data
prices = yf.download(ETFS + ["SPY"], start=START, auto_adjust=True)["Close"]
rets = prices.pct_change().fillna(0)
price_m = prices[ETFS].resample("ME").last()


In [ ]:
# Momentum signal
mom = price_m / price_m.shift(MOM) - 1
rank_pct = mom.rank(pct=True, axis=1)
signal_m = (rank_pct >= (1 - TOP_N / len(ETFS))).astype(float)
pos = signal_m.reindex(prices.index, method="ffill")
pos = pos.div(pos.sum(axis=1), axis=0).fillna(0)


In [ ]:
# Strategy returns
strategy_returns = (pos.shift() * rets[ETFS]).sum(axis=1)
strategy_returns -= pos.diff().abs().sum(axis=1) * TC
cumulative_strategy = (1 + strategy_returns).cumprod()

# SPY benchmark
spy_ret = rets["SPY"]
cum_spy = (1 + spy_ret).cumprod()


In [ ]:
# Plot cumulative returns
plt.figure(figsize=(10, 5))
plt.plot(cumulative_strategy, label="6-M Sector Rotation")
plt.plot(cum_spy, label="SPY Buy-Hold")
plt.title("6-Month Momentum Sector Rotation vs. SPY")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Performance metrics
def perf_stats(r, cr):
    a = r.dropna().to_numpy()
    if a.size == 0: return (np.nan,) * 4
    ann = a.mean() * 252
    vol = a.std(ddof=0) * np.sqrt(252)
    sr = ann / vol if vol != 0 else np.nan
    dd = np.max(np.maximum.accumulate(cr) - cr)
    return ann, vol, sr, dd

s_r, s_v, s_sr, s_dd = perf_stats(strategy_returns, cumulative_strategy)
p_r, p_v, p_sr, p_dd = perf_stats(spy_ret, cum_spy)

print(f"{'Metric':20}{'Rotation':>12}{'SPY':>12}")
print("-" * 44)
print(f"{'Ann. Return':20}{s_r:12.2%}{p_r:12.2%}")
print(f"{'Ann. Volatility':20}{s_v:12.2%}{p_v:12.2%}")
print(f"{'Sharpe Ratio':20}{s_sr:12.2f}{p_sr:12.2f}")
print(f"{'Max Drawdown':20}{s_dd:12.2%}{p_dd:12.2%}")


In [ ]:
# Attribution analysis
daily_contrib = pos.shift() * rets[ETFS]
total_contrib = daily_contrib.sum()
avg_weights = pos.mean()
attr = pd.DataFrame({
    "avg_weight": avg_weights,
    "total_return": total_contrib
}).sort_values("total_return", ascending=False)

print("=== Sector Attribution ===")
print(attr.to_string(float_format="{:,.2%}".format))


In [ ]:
# Attribution plot
fig, ax1 = plt.subplots(figsize=(8, 4))
attr["total_return"].plot.bar(ax=ax1, rot=45, position=0, width=0.4, label="Total Return")
ax2 = ax1.twinx()
attr["avg_weight"].plot.bar(ax=ax2, rot=45, position=1, width=0.4, color="C1", label="Avg Weight")
ax1.set_ylabel("Total Return")
ax2.set_ylabel("Avg Weight")
ax1.set_title("Sector Rotation Attribution")
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
plt.tight_layout()
plt.show()


In [ ]:
# Sharpe heatmap parameter sweep
lookbacks = [3, 6, 9, 12]
top_ns = [2, 3, 4, 5]
results = []

for mom in tqdm(lookbacks, desc="Lookbacks"):
    mom_ret = price_m / price_m.shift(mom) - 1
    rank_pct = mom_ret.rank(pct=True, axis=1)

    for top_n in top_ns:
        signal_m = (rank_pct >= (1 - top_n / len(ETFS))).astype(float)
        pos = signal_m.reindex(prices.index, method="ffill")
        pos = pos.div(pos.sum(axis=1), axis=0).fillna(0)
        strategy_returns = (pos.shift() * rets[ETFS]).sum(axis=1)
        strategy_returns -= pos.diff().abs().sum(axis=1) * TC
        cum = (1 + strategy_returns).cumprod()

        ann_ret = strategy_returns.mean() * 252
        ann_vol = strategy_returns.std(ddof=0) * np.sqrt(252)
        sharpe = ann_ret / ann_vol if ann_vol else np.nan
        max_dd = (cum.cummax() - cum).max()

        results.append({
            "lookback": mom,
            "top_n": top_n,
            "Ann Return": ann_ret,
            "Ann Vol": ann_vol,
            "Sharpe": sharpe,
            "Max Drawdown": max_dd
        })

df_res = pd.DataFrame(results)
df_pivot = df_res.pivot(index="lookback", columns="top_n", values="Sharpe")


In [ ]:
# Plot heatmap
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(figsize=(6, 4))
data = df_pivot.values
cax = ax.imshow(data, origin="lower", aspect="auto", cmap="viridis")
fig.colorbar(cax, label="Sharpe Ratio")

ax.set_xticks(np.arange(len(df_pivot.columns)))
ax.set_xticklabels(df_pivot.columns)
ax.set_yticks(np.arange(len(df_pivot.index)))
ax.set_yticklabels(df_pivot.index)
ax.set_xlabel("Top N ETFs")
ax.set_ylabel("Momentum Lookback (months)")
ax.set_title("Sharpe Ratio Heatmap")

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        ax.text(j, i, f"{data[i,j]:.2f}", ha="center", va="center",
                color="white" if data[i,j] < data.max()/2 else "black")

best = df_res.loc[df_res["Sharpe"].idxmax()]
bi = df_pivot.index.get_loc(best["lookback"])
bj = df_pivot.columns.get_loc(best["top_n"])
rect = Rectangle((bj - 0.5, bi - 0.5), 1, 1, fill=False, edgecolor="red", linewidth=2)
ax.add_patch(rect)

plt.tight_layout()
plt.show()
